# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [37]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [17]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [16]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [ ]:
# what is CLT...
# Ho= there is no difference between the two groups
# H1= there are differenence between the two groups

In [22]:
w = data[data.race=='w']
b = data[data.race=='b']

In [7]:
# Your solution to Q3 here

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

In [39]:
# here we reject the null hypothesis, meaning these two groups have difference
stats.ttest_ind(w.call,b.call,equal_var = False)
# Actually I think here is wrong....
# CI,margin error of w

mean_w, sigma_w = np.mean(w.call), np.std(w.call)
high_w, low_w = stats.norm.interval(0.95, loc=mean_w, scale=sigma_w)
print('confidence level = ','(', low_w ,',', high_w ,')')
margin_error_w=low_w-mean_w
print(margin_error_w)
# CI,margin error of b
mean_b, sigma_b = np.mean(b.call), np.std(b.call)
high_b, low_b = stats.norm.interval(0.95, loc=mean_b, scale=sigma_b)
print('confidence level = ','(', low_b ,',', high_b ,')')
margin_error_b=low_b-mean_b
print(margin_error_b)


confidence level =  ( 0.675256945629 , -0.482238464494 )
0.578747705062
confidence level =  ( 0.545841612784 , -0.416888841358 )
0.481365227071


In [42]:
# here we reject the null hypothesis, meaning these two groups have difference
stats.ttest_ind(w.call,b.call,equal_var = False)

Ttest_indResult(statistic=4.1147052908617514, pvalue=3.9429415136459352e-05)

In [41]:
white = sum(data[data.race=='w'].call) / len(data)
black = sum(data[data.race=='b'].call) / len(data)
w_and_b = sum(data.call) / len(data)
print(w_and_b)

0.0804928131417


In [38]:
# Actually I think here I should use A/B test, but here always an error, I think it should be the same problem as assignment1 
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2
#------------------
def draw_call_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1,data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1,perm_sample_2)

    return perm_replicates
#-------------------------
def diff_call(data_w, data_b):
    call_w = np.sum(data_w) / len(data_w)
    call_b = np.sum(data_b) / len(data_b)
    return call_w - call_b
diff_call_obs = diff_call(w.call, b.call)
call_replicates = draw_call_reps(w.call,b.call,diff_call_obs,size=10000)
p_value = np.sum(call_replicates >= diff_call_obs) / 10000

TypeError: 'numpy.float64' object is not callable